<a href="https://colab.research.google.com/github/Tarun2733/game-recommendation-system/blob/main/gameModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
torch.cuda.is_available()


True

In [4]:
!nvidia-smi


Sun May 18 05:58:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Import Pandas and Numpy

In [5]:
import pandas as pd
import numpy as np

In [6]:
games = pd.read_csv('/content/steam.csv')
tags_df = pd.read_csv('/content/steamspy_tag_data.csv')
desc_df = pd.read_csv('/content/steam_description_data.csv')
media_df = pd.read_csv('/content/steam_media_data.csv')



In [7]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19


In [8]:
tags_df.head(1)

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550


In [9]:
desc_df.head(1)

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [10]:
desc_df = desc_df.rename(columns={'steam_appid': 'appid'})

In [11]:
desc_df.head(1)

,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [12]:
media_df.head(1)

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN


In [13]:
games = games.merge(desc_df,on = 'appid')

In [14]:
games.shape

(27075, 21)

In [15]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


Select the columns that will be used:
appid,
name,
developer,
categories,
genre,
steamspy_tags,



In [16]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   appid                 27075 non-null  int64  
 1   name                  27075 non-null  object 
 2   release_date          27075 non-null  object 
 3   english               27075 non-null  int64  
 4   developer             27074 non-null  object 
 5   publisher             27061 non-null  object 
 6   platforms             27075 non-null  object 
 7   required_age          27075 non-null  int64  
 8   categories            27075 non-null  object 
 9   genres                27075 non-null  object 
 10  steamspy_tags         27075 non-null  object 
 11  achievements          27075 non-null  int64  
 12  positive_ratings      27075 non-null  int64  
 13  negative_ratings      27075 non-null  int64  
 14  average_playtime      27075 non-null  int64  
 15  median_playtime    

In [17]:
print(games.columns.tolist())


['appid', 'name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price', 'detailed_description', 'about_the_game', 'short_description']


In [18]:
games = games[['appid', 'name','short_description','developer','platforms','steamspy_tags']]

In [19]:
games.head()

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,windows;mac;linux,Action;FPS;Multiplayer
1,20,Team Fortress Classic,One of the most popular online action games of...,Valve,windows;mac;linux,Action;FPS;Multiplayer
2,30,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,Valve,windows;mac;linux,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,Valve,windows;mac;linux,Action;FPS;Multiplayer
4,50,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,Gearbox Software,windows;mac;linux,FPS;Action;Sci-fi


In [20]:
games.isnull().sum()

,0
appid,0
name,0
short_description,0
developer,1
platforms,0
steamspy_tags,0


In [21]:
games.dropna(inplace=True)

In [22]:
games.duplicated().sum()

np.int64(0)

In [23]:
games['short_description'][0]

"Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role."

In [24]:
games['steamspy_tags'][0]

'Action;FPS;Multiplayer'

In [25]:
games['platforms'][0]

'windows;mac;linux'

In [26]:
def string_to_list(st):
  if pd.isnull(st):
    return []
  return st.lower().replace(';',' ').split()


In [27]:
games['short_description'] = games['short_description'].apply(string_to_list)

In [28]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",Valve,windows;mac;linux,Action;FPS;Multiplayer


In [29]:
games['developer'] = games['developer'].apply(string_to_list)
games['platforms'] = games['platforms'].apply(string_to_list)
games['steamspy_tags'] = games['steamspy_tags'].apply(string_to_list)


In [30]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",[valve],"[windows, mac, linux]","[action, fps, multiplayer]"


In [31]:
games['tags'] = games['short_description']+games['developer']+games['platforms']+games['steamspy_tags']

In [32]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags,tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",[valve],"[windows, mac, linux]","[action, fps, multiplayer]","[play, the, world's, number, 1, online, action..."


In [33]:
df = games[['appid','name','tags']]

In [34]:
df.head()

,appid,name,tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action..."
1,20,Team Fortress Classic,"[one, of, the, most, popular, online, action, ..."
2,30,Day of Defeat,"[enlist, in, an, intense, brand, of, axis, vs...."
3,40,Deathmatch Classic,"[enjoy, fast-paced, multiplayer, gaming, with,..."
4,50,Half-Life: Opposing Force,"[return, to, the, black, mesa, research, facil..."


In [35]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
#stemming
def stem(t):
  lst = []
  for word in t.split():
    lst.append(ps.stem(word))

  return " ".join(lst)

In [36]:
df['tags'][0]

['play',
 'the',
 "world's",
 'number',
 '1',
 'online',
 'action',
 'game.',
 'engage',
 'in',
 'an',
 'incredibly',
 'realistic',
 'brand',
 'of',
 'terrorist',
 'warfare',
 'in',
 'this',
 'wildly',
 'popular',
 'team-based',
 'game.',
 'ally',
 'with',
 'teammates',
 'to',
 'complete',
 'strategic',
 'missions.',
 'take',
 'out',
 'enemy',
 'sites.',
 'rescue',
 'hostages.',
 'your',
 'role',
 'affects',
 'your',
 "team's",
 'success.',
 'your',
 "team's",
 'success',
 'affects',
 'your',
 'role.',
 'valve',
 'windows',
 'mac',
 'linux',
 'action',
 'fps',
 'multiplayer']

In [37]:
df['tags'][1]

['one',
 'of',
 'the',
 'most',
 'popular',
 'online',
 'action',
 'games',
 'of',
 'all',
 'time,',
 'team',
 'fortress',
 'classic',
 'features',
 'over',
 'nine',
 'character',
 'classes',
 '--',
 'from',
 'medic',
 'to',
 'spy',
 'to',
 'demolition',
 'man',
 '--',
 'enlisted',
 'in',
 'a',
 'unique',
 'style',
 'of',
 'online',
 'team',
 'warfare.',
 'each',
 'character',
 'class',
 'possesses',
 'unique',
 'weapons,',
 'items,',
 'and',
 'abilities,',
 'as',
 'teams',
 'compete',
 'online',
 'in',
 'a',
 'variety',
 'of',
 'game',
 'play',
 'modes.',
 'valve',
 'windows',
 'mac',
 'linux',
 'action',
 'fps',
 'multiplayer']

In [38]:
ps.stem('loved')

'love'

In [39]:
# convert list to string
df['tags'] = df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

<ipython-input-39-68cc1262373c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)


In [40]:
df = df.copy()
df['tags'] = df['tags'].apply(stem)

In [41]:
df.shape


(27074, 3)

In [42]:
#stemmed tags
df['tags'][0]

"play the world' number 1 onlin action game. engag in an incred realist brand of terrorist warfar in thi wildli popular team-bas game. alli with teammat to complet strateg missions. take out enemi sites. rescu hostages. your role affect your team' success. your team' success affect your role. valv window mac linux action fp multiplay"

Vectorization using TF-IDF


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [44]:
tfidf = TfidfVectorizer(max_features=3000, stop_words='english')

In [45]:
matrix = tfidf.fit_transform(df['tags']).toarray()

In [46]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
tfidf.get_feature_names_out()

array(['000', '10', '100', ..., 'zombies', 'zone', 'zoo'], dtype=object)

In [48]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
similarity_matrix = cosine_similarity(matrix)

In [50]:
similarity_matrix.shape

(27074, 27074)

In [55]:
#recommend function
def recommend(game):
  idx = df[df['name'] == game].index[0]
  distances = similarity_matrix[idx]
  lst = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in lst:
    print(df.iloc[i[0]]['name'])


In [56]:
recommend('Counter-Strike')

A Small Robot Story
Wonderland Trails
LEGO® Indiana Jones™ 2: The Adventure Continues
Counter Spell
The Underground Watcher/地下监察员


In [54]:
df.head(1)

,appid,name,tags
0,10,Counter-Strike,play the world' number 1 onlin action game. en...
